In [1]:
# We've built a spam detection filter using Python and Spark!

In [2]:
import findspark
findspark.init('/home/hale/spark-2.4.3-bin-hadoop2.7/')
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [7]:
data =spark.read.csv('smsspamcollection/SMSSpamCollection', inferSchema=True, sep='\t')

In [8]:
data.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [9]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [10]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [11]:
# We create a new length feature
from pyspark.sql.functions import length

In [12]:
data = data.withColumn('length', length(data['text']))

In [13]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [14]:
##
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [15]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer

In [16]:
tokenizer = Tokenizer(inputCol='text', outputCol='token_text')
stop_remove = StopWordsRemover(inputCol='token_text', outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens', outputCol='c_vec')
idf = IDF(inputCol='c_vec', outputCol='tf_idf')
ham_spam_to_numeric = StringIndexer(inputCol='class',outputCol='label')

In [17]:
from pyspark.ml.feature import VectorAssembler

In [18]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'], outputCol='features')

In [19]:
# build the model
from pyspark.ml.classification import NaiveBayes

In [20]:
nb = NaiveBayes()

In [21]:
from pyspark.ml import Pipeline

In [41]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_numeric, tokenizer, stop_remove, count_vec, idf, clean_up])

In [42]:
cleaner = data_prep_pipe.fit(data)

In [43]:
clean_data = cleaner.transform(data)

In [44]:
clean_data.columns

['class',
 'text',
 'length',
 'label',
 'token_text',
 'stop_tokens',
 'c_vec',
 'tf_idf',
 'features']

In [46]:
clean_data = clean_data.select('label','features')

In [27]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [28]:
training, test = clean_data.randomSplit([0.7, 0.3])

In [29]:
spam_detector = nb.fit(training)

In [30]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [31]:
test_results = spam_detector.transform(test)

In [32]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,7,8...|[-794.59269830338...|[1.0,6.5870457392...|       0.0|
|  0.0|(13424,[0,1,2,41,...|[-1063.2317648836...|[1.0,3.1134180655...|       0.0|
|  0.0|(13424,[0,1,3,9,1...|[-570.10493059950...|[1.0,2.8985857000...|       0.0|
|  0.0|(13424,[0,1,4,50,...|[-832.71022596854...|[1.0,1.3004335406...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[-883.77726327325...|[1.0,1.5381991298...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[-1207.9872370288...|[1.0,1.2498502826...|       0.0|
|  0.0|(13424,[0,1,15,20...|[-685.41795150594...|[1.0,6.9899036068...|       0.0|
|  0.0|(13424,[0,1,17,19...|[-806.10075865868...|[1.0,3.3043866823...|       0.0|
|  0.0|(13424,[0,1,18,20...|[-833.57247571993...|[1.0,2.4894319284...|       0.0|
|  0.0|(13424,[0

In [33]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [34]:
acc_eval = MulticlassClassificationEvaluator()

In [36]:
acc = acc_eval.evaluate(test_results)

In [37]:
print('ACC of NB Model')
print(acc)

ACC of NB Model
0.9249634323342102
